## Quick fix final

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import spacy
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, classification_report
from sklearn.metrics import mean_squared_error


from collections import Counter
import re
import string
import tqdm
spacy.load('en_core_web_sm')
stopwords = stopwords.words('english')
# torch.backends.cudnn.enabled = False 
torch.random.manual_seed(123456)

In [2]:
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.00001
NODES = 1000

In [3]:
train_split = 70
split_train = train_split/100
embed_size = 300
train_file_name = "./dataset_{}/train_{}.csv".format(train_split,train_split) 
test_file_name = "./dataset_{}/test_{}.csv".format(train_split,train_split) 
vocab_file_name =  "./dataset_{}/vocab_{}.csv".format(train_split,train_split)
glove_filename = "../../data/glove.6B.{}d.txt".format(embed_size)
print(split_train, train_file_name, test_file_name)

0.7 ./dataset_70/train_70.csv ./dataset_70/test_70.csv


In [4]:
train = pd.read_csv(train_file_name, sep="|", index_col=0)
test = pd.read_csv(test_file_name, sep="|", index_col=0)
len(train),len(test), train.columns

(19930,
 8541,
 Index(['tweet_id', 'created_time', 'count', '1', '2', '3', '4', '5', '6',
        'user_id', 'screen_name', 'url', 'follower_count', 'title', 'content',
        'expanded_url', 'title_len', 'content_len', 'max_retweets',
        'label_log_10', 'label_mean', 'label_median', 'label_quantile',
        'label_grouped_median', 'grouped_median'],
       dtype='object'))

In [5]:
# train.loc[train.max_retweets==32870]

In [6]:
tok = spacy.load('en_core_web_sm')
max_len = -1
def tokenize(text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct_num = regex.sub(" ", text.lower()) 
#     text = " ".join([word for word in text.split() if word not in stopwords])
    # Removing the odd apostrophes
    tokens = [token for token in nopunct_num.split() if len(token)>=2 ]
    return tokens
#     text = re.sub(r"[^\x00-\x7F]+", " ", text)
#     regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
#     nopunct = regex.sub(" ", text.lower())
#     return [token.text for token in tok.tokenizer(nopunct)]
#count number of occurences of each word

counts = Counter()
for index, row in train.iterrows():
    tokenized = tokenize(row['title'])
    if max_len < len(tokenized):
        max_len = len(tokenized)
    counts.update(tokenized)
print("Max number of words in len",max_len)

Max number of words in len 24


In [7]:
#deleting infrequent words

In [8]:
# # creating vocab
# vocab2index = {"":0, "UNK":1}
# words = ["","UNK"]
# for word in counts:
#     vocab2index[word] = len(words)
#     words.append(word)
# words[:5]

In [9]:
def encode_sentence(text, vocab2index, N=max_len):
    tokenized = tokenize(text)
    encoded = np.zeros(N,dtype=int)
    enc1 = np.array([vocab2index.get(word,vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [10]:
def load_glove_vectors(glove_file=glove_filename):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file, encoding="utf8") as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

In [11]:
def get_emb_matrix(pretrained, word_counts, emb_size = embed_size):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_counts) + 2
    vocab_to_idx = {}
    vocab = ["<pad>", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["<pad>"] = 0
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_counts:
        if word in word_vecs:
            W[i] = word_vecs[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx

In [12]:
word_vecs = load_glove_vectors()
pretrained_weights, vocab, vocab2index = get_emb_matrix(word_vecs, counts)

In [13]:
with open('vocab_70.npy', 'wb') as f:
    np.save(f, vocab, allow_pickle=True)
with open('vocab2index_70.npy', 'wb') as f:
    np.save(f, vocab2index, allow_pickle=True)
with open('pretrained_weights_70.npy', 'wb') as f:
    np.save(f, pretrained_weights, allow_pickle=True)

In [ ]:
pretrained_weights, vocab, vocab2index = None, None, None
with open('vocab_70.npy', 'rb') as f:
    vocab = np.load(f)
with open('vocab2index_70.npy', 'rb') as f:
    vocab2index = np.load(f)
with open('pretrained_weights_70.npy', 'rb') as f:
    pretrained_weights = np.load(f)

In [ ]:
vocab_size = len(vocab)
pretrained_weights.shape, len(vocab), len(vocab2index)

In [ ]:
train['encoded'] = train['title'].apply(lambda x: np.array(encode_sentence(x,vocab2index)))
test['encoded'] = test['title'].apply(lambda x: np.array(encode_sentence(x,vocab2index)))

In [ ]:
X_train, y_train = train['encoded'], train['label_log_10']
X_train.head(2),y_train.head(2)

In [ ]:
X_test, y_test = test['encoded'], test['label_log_10']
X_test.head(2), y_test.head(2)

In [ ]:
class NewsDataset(Dataset):
    def __init__(self, X, Y):
            self.X = X
            self.y = Y
            
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [ ]:
train_ds = NewsDataset(X_train, y_train)
test_ds = NewsDataset(X_test, y_test)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def train_model(model):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=LEARNING_RATE)#, momentum=0.7)
#     model_fixed.load_state_dict(torch.load("./model_dropout.pt"))
#     optimizer.load_state_dict(torch.load("./optimizer_dropout.pt"))
    actual_loss = None
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose=True,
                                                     patience=3, factor=0.6)
    softmax = nn.LogSoftmax(dim=0)
    for i in tqdm.tqdm(range(EPOCHS), total=EPOCHS):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long().to(device)
            y = y.long().to(device)
            y_pred = model(x, l)
            y_hat = softmax(y_pred)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item() *y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, test_dl, i)
        scheduler.step(sum_loss)
        actual_loss = val_loss
#         print("Model Dict updated")
#         torch.save(model.state_dict(),"./model_dropout.pt" )
#         torch.save(optimizer.state_dict(),"./optimizer_dropout_adam.pt")
            
#         if i%5 == 0:
        print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl, i):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    total_preds, total_actual= [], []
    for x, y, l in valid_dl:
        x = x.long().to(device)
        y = y.long()
        y_hat = model(x, l).cpu()
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
#         print(pred.numpy())
#         print(y.numpy())
        total_preds += pred.tolist()
        total_actual += y.tolist()
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
#     if i%5 == 0:
#         print(i)
    print(classification_report(total_preds, total_actual))
    return sum_loss/total, correct/total, sum_rmse/total

In [ ]:
# vocab_size = len(words)
# print(vocab_size)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE)
# np.save('vocab2index.npy',vocab2index)
# np.save('wordlist.npy',words)

In [ ]:
x,y,l = train_ds[0]
len(train_dl), x.unsqueeze(0).shape

In [ ]:
class LSTM_fixed_len(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights):
        super().__init__()
#         self.model = nn.Sequential(*[
#             nn.Embedding(vocab_size, embedding_dim, padding_idx=0),
#             nn.Dropout(0.2),
#             nn.LSTM(embedding_dim, hidden_dim, batch_first=True),
#             nn.Linear(hidden_dim, 2)
#         ])
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(0.2)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=2, bidirectional=True)
#         self.full_connecected = [nn.Linear(hidden_dim*2, hidden_dim, bias=False),
#                                 nn.functional.relu_(),
#                                 nn.Linear(hidden_dim, )]
        self.linear = nn.Linear(hidden_dim*2, 4, bias=False)
        
    def forward(self, x, l):
#         print(x[0])
#         result = self.model(x)
#         print(result)
#         return 0
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out,( ht, ct) = self.lstm(x)
#         print(ht[-1].shape, torch.cat((ht[-2],ht[-1]), dim=1).shape)
#         print(ht.view(3, 2, 128, 256).shape)
        return self.linear(torch.cat((ht[1],ht[-1]), dim=1))

In [ ]:
model_fixed = LSTM_fixed_len(vocab_size, embed_size, 256, pretrained_weights)
model_fixed

In [ ]:
EPOCHS = 100
BATCH_SIZE = 128
LEARNING_RATE = 1e-3

In [ ]:
train_model(model_fixed.to(device))  